Here, In this code I tried to training 14 users samples from room1 as training and 7 users samples from room2 as testing
Training = 1,2,3,4,6,5,10,11,12,13,14,15,16,17
Testing = 1,2,3,2,6,3,7,8,9

In [160]:
#import libraries
import os,sys
import numpy as np
import scipy.io as scio
import tensorflow as tf
import keras
from keras.layers import Input, GRU, Dense, Flatten, Dropout, Conv2D, Conv3D, MaxPooling2D, MaxPooling3D, TimeDistributed
from keras.models import Model, load_model
import keras.backend as K
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import random
import math
import collections
import pdb

In [161]:
# Parameters
data_dir1 = 'BVPr11/'
data_dir123 = 'BVPr1123/'
data_dir4 = 'BVPr14/'
data_dir6= 'BVPr16/'
data_dir5 = 'BVP5/'
data_dir10 = 'BVP10/'
data_dir11 = 'BVP11/'
data_dir12 = 'BVP12/'
data_dir13 = 'BVP13/'
data_dir14 = 'BVP14/'
data_dir15 = 'BVP15/'
data_dir16 = 'BVP16/'
data_dir17 = 'BVP17/'               
ALL_MOTION = [1,2,3,4,5,6]
N_MOTION = len(ALL_MOTION)
T_MAX = 0
f_dropout_ratio = 0.5
n_gru_hidden_units = 512
n_batch_size = 32
f_learning_rate = 0.001
num_clients = 14
num_participants = 8
batch_size =28
num_epochs =5
num_rounds = 10
use_existing_model = False

In [162]:
# make functions to preprocess Widar Dataset
# dataset shape and size: [20,20,T]
def normalize_data(widardataset):
    # data(ndarray)=>data_norm(ndarray): [20,20,T]=>[20,20,T]
    widardataset_max = np.concatenate((widardataset.max(axis=0),widardataset.max(axis=1)),axis=0).max(axis=0)
    widardataset_min = np.concatenate((widardataset.min(axis=0),widardataset.min(axis=1)),axis=0).min(axis=0)
    if (len(np.where((widardataset_max - widardataset_min) == 0)[0]) > 0):
        return widardataset
    widardataset_max_rep = np.tile(widardataset_max,(widardataset.shape[0],widardataset.shape[1],1))
    widardataset_min_rep = np.tile(widardataset_min,(widardataset.shape[0],widardataset.shape[1],1))
    widardataset_norm = (widardataset - widardataset_min_rep) / (widardataset_max_rep - widardataset_min_rep)
    return  widardataset_norm

def zero_padding(data, T_MAX):
    # data(list)=>data_pad(ndarray): [20,20,T1/T2/...]=>[20,20,T_MAX]
    data_pad = []
    for i in range(len(data)):
        t = np.array(data[i]).shape[2]
        data_pad.append(np.pad(data[i], ((0,0),(0,0),(T_MAX - t,0)), 'constant', constant_values = 0).tolist())
    return np.array(data_pad)

def onehot_encoding(label, num_class):
    # label(list)=>_label(ndarray): [N,]=>[N,num_class]
    label = np.array(label).astype('int32')
    # assert (np.arange(0,np.unique(label).size)==np.unique(label)).prod()    # Check label from 0 to N
    label = np.squeeze(label)
    _label = np.eye(num_class)[label-1]     # from label to onehot
    return _label

def load_data(path_to_data, motion_sel):
    global T_MAX
    data = []
    label = []
    for data_root, data_dirs, data_files in os.walk(path_to_data):
        for data_file_name in data_files:

            file_path = os.path.join(data_root,data_file_name)
            try:
                widardataset = scio.loadmat(file_path)['velocity_spectrum_ro']
                label_1 = int(data_file_name.split('-')[1])
                location = int(data_file_name.split('-')[2])
                orientation = int(data_file_name.split('-')[3])
                repetition = int(data_file_name.split('-')[4])

                # Select Motion
                if (label_1 not in motion_sel):
                    continue
                
                
                # Normalization
                data_normed_1 = normalize_data(widardataset)
                
                # Update T_MAX
                if T_MAX < np.array(widardataset).shape[2]:
                    T_MAX = np.array(widardataset).shape[2]                
            except Exception:
                continue

            # Save List
            data.append(data_normed_1.tolist())
            label.append(label_1)
            
    # Zero-padding
    data = zero_padding(data, T_MAX)

    # Swap axes
    data = np.swapaxes(np.swapaxes(data, 1, 3), 2, 3)   # [N,20,20',T_MAX]=>[N,T_MAX,20,20']
    data = np.expand_dims(data, axis=-1)    # [N,T_MAX,20,20]=>[N,T_MAX,20,20,1]

    # Convert label to ndarray
    label = np.array(label)

    # data(ndarray): [N,T_MAX,20,20,1], label(ndarray): [N,N_MOTION]
    return data, label

In [163]:
# dataset loading
# Load data
client1_data, client1_label = load_data(data_dir1, ALL_MOTION)
data123, label123 = load_data(data_dir123, ALL_MOTION)
data4, label4 = load_data(data_dir4, ALL_MOTION)
data6, label6 = load_data(data_dir6, ALL_MOTION)
data5, label5 = load_data(data_dir5, ALL_MOTION)
data10, label10 = load_data(data_dir10, ALL_MOTION)
data11, label11 = load_data(data_dir11, ALL_MOTION)
data12, label12 = load_data(data_dir12, ALL_MOTION)
data13, label13 = load_data(data_dir13, ALL_MOTION)
data14, label14 = load_data(data_dir14, ALL_MOTION)
data15, label15 = load_data(data_dir15, ALL_MOTION)
data16, label16 = load_data(data_dir16, ALL_MOTION)
data17, label17 = load_data(data_dir17, ALL_MOTION)


In [165]:
re1 = (client1_data.shape[0], 36, 20, 20, 1)
re_data1 = np.resize(client1_data, re1)
print('\nLoaded dataset of Client1 ' + str(client1_label.shape[0]) + ' samples, each sized ' + str(re_data1[0,:,:].shape) + '\n')
re123 = (data123.shape[0], 36, 20, 20, 1)
re_data123 = np.resize(data123, re123)
print('\nLoaded dataset of Client124 ' + str(label123.shape[0]) + ' samples, each sized ' + str(re_data123[0,:,:].shape) + '\n')
re4 = (data4.shape[0], 36, 20, 20, 1)
re_data4 = np.resize(data4, re4)
print('\nLoaded dataset of Client124 ' + str(label4.shape[0]) + ' samples, each sized ' + str(re_data4[0,:,:].shape) + '\n')
re5 = (data5.shape[0], 36, 20, 20, 1)
re_data5 = np.resize(data5, re5)
print('\nLoaded dataset of Client124 ' + str(label5.shape[0]) + ' samples, each sized ' + str(re_data5[0,:,:].shape) + '\n')
re6 = (data6.shape[0], 36, 20, 20, 1)
re_data6 = np.resize(data6, re6)
print('\nLoaded dataset of Client124 ' + str(label6.shape[0]) + ' samples, each sized ' + str(re_data6[0,:,:].shape) + '\n')
re10 = (data10.shape[0], 36, 20, 20, 1)
re_data10 = np.resize(data10, re10)
print('\nLoaded dataset of Client124 ' + str(label10.shape[0]) + ' samples, each sized ' + str(re_data10[0,:,:].shape) + '\n')
re11 = (data11.shape[0], 36, 20, 20, 1)
re_data11 = np.resize(data11, re11)
print('\nLoaded dataset of Client124 ' + str(label11.shape[0]) + ' samples, each sized ' + str(re_data11[0,:,:].shape) + '\n')
re12 = (data12.shape[0], 36, 20, 20, 1)
re_data12 = np.resize(data12, re12)
print('\nLoaded dataset of Client124 ' + str(label12.shape[0]) + ' samples, each sized ' + str(re_data12[0,:,:].shape) + '\n')
re13 = (data13.shape[0], 36, 20, 20, 1)
re_data13 = np.resize(data13, re13)
print('\nLoaded dataset of Client124 ' + str(label13.shape[0]) + ' samples, each sized ' + str(re_data13[0,:,:].shape) + '\n')
re14 = (data14.shape[0], 36, 20, 20, 1)
re_data14 = np.resize(data14, re14)
print('\nLoaded dataset of Client124 ' + str(label14.shape[0]) + ' samples, each sized ' + str(re_data14[0,:,:].shape) + '\n')
re15 = (data15.shape[0], 36, 20, 20, 1)
re_data15 = np.resize(data15, re15)
print('\nLoaded dataset of Client124 ' + str(label15.shape[0]) + ' samples, each sized ' + str(re_data15[0,:,:].shape) + '\n')
re16 = (data16.shape[0], 36, 20, 20, 1)
re_data16 = np.resize(data16, re16)
print('\nLoaded dataset of Client124 ' + str(label16.shape[0]) + ' samples, each sized ' + str(re_data16[0,:,:].shape) + '\n')
re17 = (data17.shape[0], 36, 20, 20, 1)
re_data17 = np.resize(data17, re17)
print('\nLoaded dataset of Client124 ' + str(label17.shape[0]) + ' samples, each sized ' + str(re_data17[0,:,:].shape) + '\n')




Loaded dataset of Client1 1800 samples, each sized (36, 20, 20, 1)


Loaded dataset of Client124 7500 samples, each sized (36, 20, 20, 1)


Loaded dataset of Client124 750 samples, each sized (36, 20, 20, 1)


Loaded dataset of Client124 750 samples, each sized (36, 20, 20, 1)


Loaded dataset of Client124 750 samples, each sized (36, 20, 20, 1)


Loaded dataset of Client124 750 samples, each sized (36, 20, 20, 1)


Loaded dataset of Client124 750 samples, each sized (36, 20, 20, 1)


Loaded dataset of Client124 750 samples, each sized (36, 20, 20, 1)


Loaded dataset of Client124 750 samples, each sized (36, 20, 20, 1)


Loaded dataset of Client124 750 samples, each sized (36, 20, 20, 1)


Loaded dataset of Client124 748 samples, each sized (36, 20, 20, 1)


Loaded dataset of Client124 750 samples, each sized (36, 20, 20, 1)


Loaded dataset of Client124 750 samples, each sized (36, 20, 20, 1)



In [166]:
num_shifted_samples = 1500
shifted_data = re_data123[:num_shifted_samples]
shifted_label = label123[:num_shifted_samples]
print('\n' + str(shifted_label.shape[0]) + ' samples, each sized ' + str(shifted_data[0,:,:].shape) + '\n') #1500 samples, each sized (30, 20, 20, 1)


1500 samples, each sized (36, 20, 20, 1)



In [167]:
re_data123 = re_data123[num_shifted_samples:]
label123 = label123[num_shifted_samples:]
print('\n' + str(label123.shape[0]) + ' samples, each sized ' + str(re_data123[0,:,:].shape) + '\n') #6000 samples, each sized (30, 20, 20, 1)


6000 samples, each sized (36, 20, 20, 1)



In [168]:
combined_data = np.concatenate((re_data1, shifted_data), axis=0)
combined_label = np.concatenate((client1_label, shifted_label), axis=0)

In [169]:
[Data1,Label1]  = (combined_data,combined_label)
#client1_label = combined_label
print('\n' + str(Label1.shape[0]) + ' samples, each sized ' + str(Data1[0,:,:].shape) + '\n') #3300 samples, each sized (30, 20, 20, 1)


3300 samples, each sized (36, 20, 20, 1)



In [170]:
# Split Client2, Client3
[Data2, Data3, Label2, Label3] = train_test_split(re_data123, label123, test_size=0.5)
print('\nTrain on ' + str(Data2.shape[0]) + ' samples\n' +\
    'Test on ' + str(Label3.shape[0]) + ' samples\n')


Train on 3000 samples
Test on 3000 samples



In [171]:
data = np.concatenate((Data1, Data2, Data3, re_data4, re_data5, re_data6, re_data10, re_data11, re_data12, re_data13, re_data14, re_data15, re_data16, re_data17 ), axis=0)
print(len(data))
print('\n' + str(data.shape[0]) + ' samples, each sized ' + str(data[0,:,:].shape) + '\n') 



17548

17548 samples, each sized (36, 20, 20, 1)



In [172]:
# One-hot encoding for train data
Label1 = onehot_encoding(Label1, N_MOTION)
Label2 = onehot_encoding(Label2, N_MOTION)
Label3 = onehot_encoding(Label3, N_MOTION)
Label4 = onehot_encoding(label4, N_MOTION)
Label6 = onehot_encoding(label6, N_MOTION)
Label5 = onehot_encoding(label5, N_MOTION)
Label10 = onehot_encoding(label10, N_MOTION)
Label11 = onehot_encoding(label11, N_MOTION)
Label12 = onehot_encoding(label12, N_MOTION)
Label13 = onehot_encoding(label13, N_MOTION)
Label14 = onehot_encoding(label14, N_MOTION)
Label15 = onehot_encoding(label15, N_MOTION)
Label16 = onehot_encoding(label16, N_MOTION)
Label17 = onehot_encoding(label17, N_MOTION)

In [173]:
label = np.concatenate((Label1,Label2, Label3, Label4, Label5, Label6, Label10, Label11, Label12, Label13, Label14, Label15, Label16, Label17), axis=0)
print(len(label))
print(str(label.shape[0]))
print(label)

17548
17548
[[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]]


In [174]:
#Test dataset loading
data_dirtest = 'BVPr2test1/'
data_test, label_test = load_data(data_dirtest, ALL_MOTION)
print('\nLoaded dataset of testing ' + str(label_test.shape[0]) + ' samples, each sized ' + str(data_test[0,:,:].shape) + '\n')


Loaded dataset of testing 5994 samples, each sized (36, 20, 20, 1)



In [175]:
label_test = onehot_encoding(label_test, N_MOTION)

In [176]:
#global federated model
def assemble_model(input_shape, n_class):
    model_input = Input(shape=input_shape, dtype='float32', name='name_model_input')    # (@,T_MAX,20,20,1)

    # Feature extraction part
    x = TimeDistributed(Conv2D(16,kernel_size=(5,5),activation='relu',data_format='channels_last',\
        input_shape=input_shape))(model_input)   # (@,T_MAX,20,20,1)=>(@,T_MAX,16,16,16)
    x = TimeDistributed(MaxPooling2D(pool_size=(2,2)))(x)    # (@,T_MAX,16,16,16)=>(@,T_MAX,8,8,16)
    x = TimeDistributed(Flatten())(x)   # (@,T_MAX,8,8,16)=>(@,T_MAX,8*8*16)
    x = TimeDistributed(Dense(64,activation='relu'))(x) # (@,T_MAX,8*8*16)=>(@,T_MAX,64)
    x = TimeDistributed(Dropout(f_dropout_ratio))(x)
    x = TimeDistributed(Dense(64,activation='relu'))(x) # (@,T_MAX,64)=>(@,T_MAX,64)
    x = GRU(n_gru_hidden_units,return_sequences=False)(x)  # (@,T_MAX,64)=>(@,128)
    x = Dropout(f_dropout_ratio)(x)

    # Communication part
    x = Flatten()(x)    # (@,128)=>(@,128)
    x = Dense(32, activation='relu')(x) # (@,128)=>(@,32)

    # Output part
    model_output = Dense(n_class, activation='softmax', name='name_model_output')(x)  # (@,32)=>(@,n_class)

    # Model compiling
    model = Model(inputs=model_input, outputs=model_output)
    model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=f_learning_rate),
                    loss='categorical_crossentropy',
                    metrics=['accuracy']
                )
    return model


In [177]:
# Create a copy of the model for aggregation
global_model = assemble_model(input_shape=(T_MAX, 20, 20, 1), n_class=N_MOTION)
global_model_copy = keras.models.clone_model(global_model)
global_model_copy.set_weights(global_model.get_weights())

In [178]:
import numpy as np

data_allocation = {1: 3300, 2: 3000, 3: 3000, 4: 750, 5: 750, 6: 750, 
                   7: 750, 8: 750, 9:750, 10:750, 11:750, 12:748, 13: 750, 14: 750}

# Get the number of samples in data
total_samples = data.shape[0]

# Assign the samples to clients consecutively
client_data = []
client_label = []
current_index = 0
for client, num_samples in data_allocation.items():
    client_data.append(data[current_index:current_index + num_samples])
    client_label.append(label[current_index:current_index + num_samples])
    current_index += num_samples

# Print the dimensions of the assigned samples for each client
for i in range(len(client_data)):
    print("Client {}: Data shape - {}, Label shape - {}".format(i+1, client_data[i].shape, client_label[i].shape))

Client 1: Data shape - (3300, 36, 20, 20, 1), Label shape - (3300, 6)
Client 2: Data shape - (3000, 36, 20, 20, 1), Label shape - (3000, 6)
Client 3: Data shape - (3000, 36, 20, 20, 1), Label shape - (3000, 6)
Client 4: Data shape - (750, 36, 20, 20, 1), Label shape - (750, 6)
Client 5: Data shape - (750, 36, 20, 20, 1), Label shape - (750, 6)
Client 6: Data shape - (750, 36, 20, 20, 1), Label shape - (750, 6)
Client 7: Data shape - (750, 36, 20, 20, 1), Label shape - (750, 6)
Client 8: Data shape - (750, 36, 20, 20, 1), Label shape - (750, 6)
Client 9: Data shape - (750, 36, 20, 20, 1), Label shape - (750, 6)
Client 10: Data shape - (750, 36, 20, 20, 1), Label shape - (750, 6)
Client 11: Data shape - (750, 36, 20, 20, 1), Label shape - (750, 6)
Client 12: Data shape - (748, 36, 20, 20, 1), Label shape - (748, 6)
Client 13: Data shape - (750, 36, 20, 20, 1), Label shape - (750, 6)
Client 14: Data shape - (750, 36, 20, 20, 1), Label shape - (750, 6)


In [179]:
#to select some clients instead of defined full clients(Client number=15, number of client per iteration=5)
num_clients_per_iteration = 5  # Customize this to control the number of clients per iteration

non_empty_clients = [i for i, data in enumerate(client_data) if data.shape[0] > 0]
non_empty_client_data = [client_data[i] for i in non_empty_clients]
non_empty_client_label = [client_label[i] for i in non_empty_clients]

for i in range(num_rounds):
    shuffled_indices = random.sample(non_empty_clients, num_clients_per_iteration)
    print('Round {}/{}'.format(i+1, num_rounds))
    client_weights_list = []
    for j in shuffled_indices:
        client_num = non_empty_clients[j]
        print('Training client {}/{}'.format(j+1, len(non_empty_clients)))
        client_model = assemble_model(input_shape=(T_MAX, 20, 20, 1), n_class=N_MOTION)
        client_model.fit({'name_model_input': non_empty_client_data[client_num]},
                         {'name_model_output': non_empty_client_label[client_num]},
                         batch_size=batch_size, epochs=num_epochs, verbose=1)
        client_weights = client_model.get_weights()
        client_weights_list.append(client_weights)
    global_weights = [np.mean([client_weights_list[k][l] for k in range(len(shuffled_indices))], axis=0)
                      for l in range(len(client_weights_list[0]))]
    global_model_copy.set_weights(global_weights)

Round 1/10
Training client 11/14
Epoch 1/5
27/27 [==============================] - 4s 118ms/step - loss: 1.7989 - accuracy: 0.1453
Epoch 2/5
27/27 [==============================] - 4s 163ms/step - loss: 1.7875 - accuracy: 0.1747
Epoch 3/5
27/27 [==============================] - 5s 167ms/step - loss: 1.8122 - accuracy: 0.3453
Epoch 4/5
27/27 [==============================] - 4s 164ms/step - loss: 1.2849 - accuracy: 0.4827
Epoch 5/5
27/27 [==============================] - 4s 160ms/step - loss: 1.2997 - accuracy: 0.4840
Training client 7/14
Epoch 1/5
27/27 [==============================] - 5s 160ms/step - loss: 1.7963 - accuracy: 0.1587
Epoch 2/5
27/27 [==============================] - 4s 164ms/step - loss: 1.7942 - accuracy: 0.1747
Epoch 3/5
27/27 [==============================] - 4s 160ms/step - loss: 1.7951 - accuracy: 0.1587
Epoch 4/5
27/27 [==============================] - 5s 170ms/step - loss: 1.7900 - accuracy: 0.1760
Epoch 5/5
27/27 [==============================] - 4s 1

In [180]:
global_model_copy.compile(optimizer=keras.optimizers.RMSprop(learning_rate=f_learning_rate),
                    loss='categorical_crossentropy',
                    metrics=['accuracy']
                )
# Evaluate the global model on the test data
global_model_copy.evaluate({'name_model_input': data_test}, {'name_model_output': label_test})

188/188 [==============================] - 22s 115ms/step - loss: 1.7919 - accuracy: 0.1458


[1.7918908596038818, 0.1458124816417694]